In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import random

In [ ]:
def dis(a,b,ax=1):
    return np.linalg.norm(a-b,axis=ax);   

In [ ]:
def generatenewFeature():
    
    with open("/Users/liz/Desktop/CS224W/hw3-bundle/node2vec/emb/investor_30walk.emb") as file:
        lines = file.readlines()
        head = lines[0]
        numOfNodes, numOfDim = head.split(' ')
    #     print numOfNodes,numOfDim
    #     439,128

        feature_matrix = np.zeros((int(numOfNodes),int(numOfDim)))
        for line in lines[1:]:
            singleNode = line.split(' ')
            node = int(singleNode[0])
            new_node = old_new_idx_dict[node]
            feature_vector = []
            for dim in singleNode[1:]:
                dim = float(dim)
                feature_vector.append(dim)
            feature_matrix[new_node] = feature_vector
    return feature_matrix

In [ ]:
feature_matrix = generatenewFeature()
print feature_matrix

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(feature_matrix)

In [ ]:
result = kmeans.labels_
print result

In [ ]:
import pandas as pd

In [ ]:
feat_cols = [ 'pixel'+str(i) for i in range(feature_matrix.shape[1]) ]

df = pd.DataFrame(feature_matrix,columns=feat_cols)
df['label'] = result
df['label'] = df['label'].apply(lambda i: str(i))

feature_matrix_walk10, result = None, None

print 'Size of the dataframe: {}'.format(df.shape)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(df[feat_cols].values)

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]

print 'Explained variation per principal component: {}'.format(pca.explained_variance_ratio_)

In [ ]:
from ggplot import *

chart = ggplot( df.loc[:,:], aes(x='pca-one', y='pca-two', z='pca-three',color='label') ) \
        + geom_point(size=75,alpha=0.8) \
        + ggtitle("First and second Principal Components")
chart

In [ ]:
edge_weights_file = "data/investor_network_undirected_weights.txt"
f = open(edge_weights_file, "r")
se = set()
for line in f:
    s,d,w = [int(i) for i in line.strip().split()]
    se.add(s)
    se.add(d)
new_old_idx_arr = sorted(list(se))
old_new_idx_dict = {}
for new,old in enumerate(new_old_idx_arr):
    old_new_idx_dict[old] = new

In [ ]:
result_orig_idx = {}
for new, c in enumerate(result):
    old = new_old_idx_arr[new]
    result_orig_idx[old] = c
print result_orig_idx

In [ ]:
result_community = {}
for k,v in result_orig_idx.iteritems():
    if v not in result_community:
        result_community[v] = []
    result_community[v].append(k)
print result_community 

In [ ]:
for v in result_community:
    print "community %d has %d nodes"% (v,len(result_community[v]))

In [ ]:
import community_layout as layout
pos = layout.community_layout(G_nx, result_orig_idx)
nx.draw_networkx_nodes(G_nx, pos, node_size=50, node_color=list(result_orig_idx.values()), cmap = plt.get_cmap('jet'))
nx.draw_networkx_edges(G_nx, pos, alpha=0.1)
# plt.savefig('node2vec_8.png')
plt.show()

In [ ]:
with open("node2vec_12_10_result.txt",'w') as file:
    with open("data/investor_list.txt",'r') as f1:
        for v in result_community:
            file.write("***********************************************************"+'\n')
            file.write("community %d has %d nodes"% (v,len(result_community[v])) + '\n')
            for idx in result_community[v]: 
                f1.seek(0)
                file.write(str(idx) +' '+ str(f1.readlines()[idx]))

In [ ]:
G_nx = nx.Graph()
node_list = []
investor_list = []
node_size_list = []
weights_dict = {}
transactions_startup2vc = {}

In [ ]:
def construct_graph(network):
    if network == "investor":
        node_list_file = "data/investor_list.txt"
        edge_list_file = "data/investor_network_undirected_unweighted.txt"

    elif network == "startup":
        node_list_file = "data/startup_list.txt"
        edge_list_file = "data/startup_network_undirected_unweighted.txt"

    f = open(node_list_file, "r")
    for line in f:
        name, w = line.strip().rsplit(' ', 1)
        node_list.append(name)
        node_size_list.append(int(w))

    if network == "startup":
        f = open("data/investor_list.txt", "r")
        for line in f:
            name, w = line.strip().rsplit(' ', 1)
            investor_list.append(name)

        f = open("data/transactions.csv", "r")
        csv_reader = csv.reader(f, delimiter=",", quotechar='"')
        for vc, startup, round, time in csv_reader:
            transactions_startup2vc[startup] = vc

    global G_nx
    if edge_list_file.endswith("txt"):
        f = open(edge_list_file, "r")
        f.readline()
        f.readline()
        f.readline()
        for line in f:
            s, d = [int(i) for i in line.strip().split()]
            G_nx.add_node(s, name=node_list[s])
            G_nx.add_node(d, name=node_list[d])

            G_nx.add_edge(s, d)

    print("nodes", G_nx.number_of_nodes(), "edges", G_nx.number_of_edges())

In [ ]:
construct_graph("investor")